In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops

## Try to train and Cross validate without creating a seperate crossvalidation graph:

The objective of this section is to just check if the following approach has the same weight for the 1 iteration of training and the crossvalidation stage. If that is the case then we know that the weights are not changed or updted because of the crossvalidation data.

We also proceed the above state by optimizing with the function with cross validation data. If the weith changes than we know that infact the first stage was correct.

In [7]:
batch_size = 4
dim = 3
hidden_units1 = 4
hidden_units2 = 4

def linearActivation1(scope, x):
    with tf.variable_scope(scope):
        w = tf.get_variable(dtype=tf.float32, initializer=tf.random_normal_initializer(seed=432), shape=[dim, hidden_units1], name="w1")
        b = tf.get_variable(dtype = tf.float32, initializer=tf.zeros(dtype=tf.float32, shape=[hidden_units1]), name="b1" )
        
        return tf.nn.tanh(tf.matmul(x, w) + b)
    
def linearActivation2(scope, x):
    with tf.variable_scope(scope):
        w = tf.get_variable(dtype=tf.float32, initializer=tf.random_normal_initializer(seed=989), shape=[hidden_units1, hidden_units2], name="w2")
        b = tf.get_variable(dtype = tf.float32, initializer=tf.zeros(dtype=tf.float32, shape=[hidden_units2]), name="b2" )
        
        return tf.nn.tanh(tf.matmul(x, w) + b)
    
    
ops.reset_default_graph()
sess = tf.Session()
with sess.as_default():
    x = tf.placeholder(dtype=tf.float32, shape=[None, dim], name="x")
    y = tf.placeholder(dtype=tf.int32, shape=[None], name="y")
    
    logits1 = linearActivation1("Layer1", x)
#     print (logits1.get_shape().as_list())
    logits2 = linearActivation2("Layer2", logits1)
    
    
    #################
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits2, labels=y, name="xentropy")
    lossCE = tf.reduce_mean(cross_entropy) 
    optimizer = tf.train.GradientDescentOptimizer(1e-5).minimize(lossCE)
        
    #################
    
    print ('Begin training Begin training Begin training Begin training Begin training ')
    print ('')
    
    np.random.seed(398)
    trainData = np.random.randn(batch_size, dim)
    trainLabels = np.random.randint(0, hidden_units2, size=batch_size)

    
    # RUN SESSION:
    sess.run(tf.global_variables_initializer())
    
    # When Training
    _, loss, cent = sess.run([optimizer, lossCE, cross_entropy], feed_dict={x:trainData, y:trainLabels})
    

    print ('############### Check the weights after the training batch (compare the results to previous example): \n')
    print ('The loss after one iteration is: ', loss)
    print ('')
    variableList = [v.name for v in tf.trainable_variables()]
    print ('The variable list is: ', variableList)
    print ('')
    print ('The layer 1 weight is: \n', sess.run([v for v in tf.trainable_variables() if v.name == 'Layer1/w1:0']))
    print ('')
    print ('The layer 1 bias is: \n', sess.run([v for v in tf.trainable_variables() if v.name == 'Layer1/b1:0']))
    print ('')
    print ('The layer 2 weight is: \n', sess.run([v for v in tf.trainable_variables() if v.name == 'Layer2/w2:0']))
    print ('')
    print ('The layer 2 bias is: \n', sess.run([v for v in tf.trainable_variables() if v.name == 'Layer2/b2:0']))
    print ('')
    print ('')
    
    # When Cross Validation (Since we dont initiate the computation graph for optimization, the weights dont change)
    np.random.seed(778)
    validData = np.random.randn(batch_size, dim)
    validLabel = np.random.randint(0, hidden_units2, size=batch_size)
    
    cent = sess.run([ cross_entropy], feed_dict={x:validData, y:validLabel})
    

    print ('############### Check if the weights are the same after the validation batch (compare the results to weights after training example): \n')
    print ('The loss after validation iteration is: ', loss)
    print ('')
    variableList = [v.name for v in tf.trainable_variables()]
    print ('The variable list is: ', variableList)
    print ('')
    print ('The layer 1 weight is: \n', sess.run([v for v in tf.trainable_variables() if v.name == 'Layer1/w1:0']))
    print ('')
    print ('The layer 1 bias is: \n', sess.run([v for v in tf.trainable_variables() if v.name == 'Layer1/b1:0']))
    print ('')
    print ('The layer 2 weight is: \n', sess.run([v for v in tf.trainable_variables() if v.name == 'Layer2/w2:0']))
    print ('')
    print ('The layer 2 bias is: \n', sess.run([v for v in tf.trainable_variables() if v.name == 'Layer2/b2:0']))
    
    # When Cross Validation with optimization and loss (This should change the weights)

    _, loss, cent = sess.run([optimizer, lossCE, cross_entropy], feed_dict={x:validData, y:validLabel})
    
    print ('############### Check if the weights change after using the optimizer with validation dataset : \n')
    print ('The loss after validation iteration is: ', loss)
    print ('')
    variableList = [v.name for v in tf.trainable_variables()]
    print ('The variable list is: ', variableList)
    print ('')
    print ('The layer 1 weight is: \n', sess.run([v for v in tf.trainable_variables() if v.name == 'Layer1/w1:0']))
    print ('')
    print ('The layer 1 bias is: \n', sess.run([v for v in tf.trainable_variables() if v.name == 'Layer1/b1:0']))
    print ('')
    print ('The layer 2 weight is: \n', sess.run([v for v in tf.trainable_variables() if v.name == 'Layer2/w2:0']))
    print ('')
    print ('The layer 2 bias is: \n', sess.run([v for v in tf.trainable_variables() if v.name == 'Layer2/b2:0']))
sess.close()

Begin training Begin training Begin training Begin training Begin training 

############### Check the weights after the training batch (compare the results to previous example): 

The loss after one iteration is:  2.11358

The variable list is:  ['Layer1/w1:0', 'Layer1/b1:0', 'Layer2/w2:0', 'Layer2/b2:0']

The layer 1 weight is: 
 [array([[-2.11875343,  1.11074066, -1.60225117, -1.04866743],
       [ 1.07435846, -0.37837502, -0.39874262,  1.21538198],
       [ 0.70911151, -0.10536171, -1.40221608, -0.44625875]], dtype=float32)]

The layer 1 bias is: 
 [array([  1.15548055e-07,   7.86918690e-07,   5.52410668e-08,
         5.78365530e-07], dtype=float32)]

The layer 2 weight is: 
 [array([[-2.41577721, -0.75244588,  2.13699627,  0.52897501],
       [-0.88196021, -1.1298064 , -0.13271846,  1.08850992],
       [-1.20912242, -0.32934937,  0.7239632 , -0.67482013],
       [ 0.30861691, -1.04130256,  0.30698499,  0.04391584]], dtype=float32)]

The layer 2 bias is: 
 [array([  6.90829935e-08,